# Análise de Convergência-Divergência da Média Móvel (MACD)
O indicador MACD é uma técnica para avaliar início de tendências, mais avançado que as médias móveis de curto e longo prazo, pois considera três médias moveis sendo todas elas exponenciais.<br />
A partir das três médias geramos duas séries de tendências para avaliar seus cruzamentos e os sinais de negociação.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
import plotly.subplots as pls
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Acoes = "CPLE3.SA"
Periodo_Inicio = datetime.datetime(2019,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
Dados = pdr.get_data_yahoo(Acoes, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados

### Análise dos Preços
Candlestick da série de preços do ativo.

In [ ]:
SeriePrecos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados.Close))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

In [ ]:
CandlePrecos = plg.Figure(plg.Candlestick(x=Dados.index,
                                          open=Dados.Open,
                                          high=Dados.High,
                                          low=Dados.Low,
                                          close=Dados.Close))
CandlePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(CandlePrecos, "Preços")

### Média Móvel Exponencial (EMA)
Cálculo da série de médias móveis exponencial.<br />
Com a série de EMA é possível avaliar a tendência de alta ou de queda do ativo, dando um peso mais alto para os preços mais recentes.

In [ ]:
def CalculaEMA(JanelaDeTempo, Precos):
    FatorK = (2 / (JanelaDeTempo+1))
    MA = Precos.rolling(window=JanelaDeTempo).mean().dropna()
    EMA = pd.DataFrame(index=MA.index)
    EMA["Preco"] = Precos
    EMA["MA"] = MA
    EMA["EMA"] = np.NaN
    EMA.EMA[0] = EMA.MA[1]

    for i in range(1, len(EMA)):
        EMA.EMA[i] = (EMA.Preco[i] * FatorK) + ((1-FatorK) * EMA.EMA[i-1])
        
    return EMA

### MACD e Sinais
Análise do cruzamento das médias móveis com o indicador MACD. Por padrão no mercado financeiro, usa-se os períodos de 9 (MACD), 12 (EMA) e 26 (EMA).
- Média curta (MACD) cruzando a longa para cima = Sinal para Compra.
- Média curta (MACD) cruzando a longa para baixo = Sinal para Venda.

In [ ]:
EMA_12 = CalculaEMA(12, Dados.Close)

In [ ]:
EMA_26 = CalculaEMA(26, Dados.Close)

In [ ]:
MACD = EMA_12.EMA - EMA_26.EMA

In [ ]:
MA_Sinal = CalculaEMA(9, MACD.dropna()).EMA

In [ ]:
Serie = Dados[MA_Sinal.index[0]:]

In [ ]:
SerieComMediaMovel = pls.make_subplots(rows=2, cols=1)
SerieComMediaMovel.add_trace(plg.Candlestick(x=Serie.index,
                                            open=Serie.Open,
                                            high=Serie.High,
                                            low=Serie.Low,
                                            close=Serie.Close,
                                            name="Preços"), 1, 1)
SerieComMediaMovel.add_trace(plg.Scatter(x=MACD.index,
                                        y=MACD,
                                        name="MACD",
                                        opacity=0.5), 2, 1)
SerieComMediaMovel.add_trace(plg.Scatter(x=MA_Sinal.index,
                                        y=MA_Sinal,
                                        name="Sinal",
                                        opacity=0.5), 2, 1)
Plot_Show(SerieComMediaMovel, "Preços & MACD")

### Histograma MACD
Mantendo o conceito de avaliar cruzamentos do MACD vs Sinal, uma visualizar alternativa é com uso do histograma.<br />
Com o histograma avalia-se os sinais:
- Acima de zero, sinal de compra.
- Abaixo de zero, sinal de venda.

In [ ]:
Hist_MACD = MACD - MA_Sinal

In [ ]:
SerieComMediaMovel = pls.make_subplots(rows=2, cols=1)
SerieComMediaMovel.add_trace(plg.Candlestick(x=Serie.index,
                                             open=Serie.Open,
                                             high=Serie.High,
                                             low=Serie.Low,
                                             close=Serie.Close,
                                             name="Preços"), 1, 1)
SerieComMediaMovel.add_trace(plg.Scatter(x=Hist_MACD.index,
                                         y=Hist_MACD,
                                         fill="tozeroy",
                                         name="MACD"), 2, 1)
Plot_Show(SerieComMediaMovel, "Preços & MACD")